# **Kaggle, competizione "Spaceship Titanic"**



Import delle librerie necessarie(principalmente pandas per i dataframe, numpy per le operazioni matriciali e sklearn per tutta la parte dei modelli)

In [ ]:
import sklearn as sk
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import roc_curve, roc_auc_score, classification_report, accuracy_score, confusion_matrix
import pandas as pd
import numpy as np

Carico dataset di test e train tramite google drive(avrei potuto anche caricarli direttamente da kaggle con la libreria 'kaggle', ma essendo contenuti di dimensioni ho preferito salvarli in locale)

In [ ]:
orig_train_dataset = pd.read_csv("./drive/MyDrive/spaceshipTitanic/train.csv")
orig_test_dataset = pd.read_csv("./drive/MyDrive/spaceshipTitanic/test.csv")

Faccio una copia del dataset di test originale e ne estraggo la colonna dei risultati attesi(ground_truth). Poi, elimino le colonne che per me non sono significative.

In [ ]:
train_dataset = orig_train_dataset.copy()

train_y = train_dataset.pop("Transported")

train_dataset.drop(["PassengerId", "Name", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"], axis = 1, inplace = True)

Rimpiazzo i valori booleani "qualificativi" con valori quantitativi che ne rispettino la distribuzione, in modo da omologare le distribuzioni dei dati

In [ ]:
train_dataset.CryoSleep.replace(False, 0, inplace = True)
train_dataset.CryoSleep.replace(True, 1, inplace = True)

train_dataset.VIP.replace(False, 0, inplace = True)
train_dataset.VIP.replace(True, 1, inplace = True)

"A mano" faccio lo stesso procedimento della trasformazione dei True a 1 e dei False a 0, ma con i valori "qualificativi" della colonna "HomePlanet", dato che sono solo tre valori(esclusi i nan). I nan poi li tratto sempre manualmente a fine cella

In [ ]:
planet_values = train_dataset.HomePlanet.unique()
print(planet_values)

train_dataset["HomePlanet"].replace("Europa", 0, inplace = True)
train_dataset["HomePlanet"].replace("Earth", 1, inplace = True)
train_dataset["HomePlanet"].replace("Mars", 2, inplace = True)

train_dataset.HomePlanet = train_dataset.HomePlanet.fillna(4)

train_dataset

['Europa' 'Earth' 'Mars' nan]


,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP
0,0.0,0.0,B/0/P,TRAPPIST-1e,39.0,0.0
1,1.0,0.0,F/0/S,TRAPPIST-1e,24.0,0.0
2,0.0,0.0,A/0/S,TRAPPIST-1e,58.0,1.0
3,0.0,0.0,A/0/S,TRAPPIST-1e,33.0,0.0
4,1.0,0.0,F/1/S,TRAPPIST-1e,16.0,0.0
...,...,...,...,...,...,...
8688,0.0,0.0,A/98/P,55 Cancri e,41.0,1.0
8689,1.0,1.0,G/1499/S,PSO J318.5-22,18.0,0.0
8690,1.0,0.0,G/1500/S,TRAPPIST-1e,26.0,0.0
8691,0.0,0.0,E/608/S,55 Cancri e,32.0,0.0


Faccio la stessa cosa di sopra con i valori della cabina, con l'unico problema che sono un paio di valori in più(circa 6500). Per questo, lascio gestire la cosa ad un for.

In [ ]:
train_dataset["Cabin"] = train_dataset["Cabin"].fillna("[0]")
cabin_values = train_dataset['Cabin'].unique()
cabin_values

for cabin_value in cabin_values:
  train_dataset.Cabin.replace(cabin_value, float(np.where(cabin_values == cabin_value)[0][0]), inplace = True)

train_dataset

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP
0,0.0,0.0,0.0,TRAPPIST-1e,39.0,0.0
1,1.0,0.0,1.0,TRAPPIST-1e,24.0,0.0
2,0.0,0.0,2.0,TRAPPIST-1e,58.0,1.0
3,0.0,0.0,2.0,TRAPPIST-1e,33.0,0.0
4,1.0,0.0,3.0,TRAPPIST-1e,16.0,0.0
...,...,...,...,...,...,...
8688,0.0,0.0,6557.0,55 Cancri e,41.0,1.0
8689,1.0,1.0,6558.0,PSO J318.5-22,18.0,0.0
8690,1.0,0.0,6559.0,TRAPPIST-1e,26.0,0.0
8691,0.0,0.0,6560.0,55 Cancri e,32.0,0.0


In [ ]:
train_dataset["Destination"] = train_dataset["Destination"].fillna("[0]")
destination_values = train_dataset.Destination.unique()
print(destination_values)

for destination_value in destination_values:
  train_dataset.Destination.replace(destination_value, np.where(destination_values == destination_value)[0][0], inplace = True)

train_dataset

['TRAPPIST-1e' 'PSO J318.5-22' '55 Cancri e' '[0]']


,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP
0,0.0,0.0,0.0,0,39.0,0.0
1,1.0,0.0,1.0,0,24.0,0.0
2,0.0,0.0,2.0,0,58.0,1.0
3,0.0,0.0,2.0,0,33.0,0.0
4,1.0,0.0,3.0,0,16.0,0.0
...,...,...,...,...,...,...
8688,0.0,0.0,6557.0,2,41.0,1.0
8689,1.0,1.0,6558.0,1,18.0,0.0
8690,1.0,0.0,6559.0,0,26.0,0.0
8691,0.0,0.0,6560.0,2,32.0,0.0


Ultimo controllo sui "nan"

In [ ]:
train_dataset[np.isnan(train_dataset) == True] = 0

Dichiaro il classificatore che voglio usare, in questo caso un semplice classificatore di regressione logistica(ricordiamo che la variabile dipendente target ha come campo d'esistenza [0,1], quindi un classificatore logistico andrà già bene).
Inoltre creo anche uno scaler per normalizzare i dati e portarli ad una distribuzione Gaussiana standard(di fatto, normale).


In [ ]:
classifier = LogisticRegression()
scaler = StandardScaler()

model = Pipeline([("scaler", scaler), ("classifier", classifier)])

Cella bonus con un altro classificatore. Utile per la cross-validation del modello. Ovviamente ri-dichiarare la variabile 'model' cancella il vecchio modello, quindi è meglio salvarsi i vecchi risulatati o salvare il modello in un'altra variabile per confrontare i risultati run-time.

In [ ]:
#model = GradientBoostingClassifier(n_estimators = 500, subsample=0.9)

Fit del modello per addestrarlo, ovviamente sui dati di train.

In [ ]:
model.fit(train_dataset, train_y)

Calcolo dei risultati. I principali che mi interessano sono due:

1.   **train_y_hat**: che non è altro che la predizione della variabile target, ovvero 0 o 1
2.   **train_y_hat_prob**: è uno step precedente alla predizione completa, ovvero quando il modello assegna solo delle probabilità ai valori della variabile target(cioè prima che effettui il softmax sulle uniche due probabilità '*P(y_hat = 1 | X)*' e '*1 - P(y_hat = 1 | X)*' )



In [ ]:
train_y_hat = model.predict(train_dataset)
train_y_hat_prob = model.predict_proba(train_dataset)


acc_score = accuracy_score(train_y_hat, train_y)

acc_score

0.7839641090532612

78% di precisione sui dati di train, non male. Vediamo quanto scenderà questa percentuale con i dati di test.
Prima di dargli in pasto i dati di test però dobbiamo pre-processarli proprio come i dati di train, per non incappare in risultati falsati. Il processo è speculare a quello fatto precedentemente, solo su un df diverso.

In [ ]:
test_dataset = orig_test_dataset.copy()

test_dataset.drop(["PassengerId", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck", "Name"], axis = 1, inplace = True)
test_dataset

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP
0,Earth,True,G/3/S,TRAPPIST-1e,27.0,False
1,Earth,False,F/4/S,TRAPPIST-1e,19.0,False
2,Europa,True,C/0/S,55 Cancri e,31.0,False
3,Europa,False,C/1/S,TRAPPIST-1e,38.0,False
4,Earth,False,F/5/S,TRAPPIST-1e,20.0,False
...,...,...,...,...,...,...
4272,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False
4273,Earth,False,NaN,TRAPPIST-1e,42.0,False
4274,Mars,True,D/296/P,55 Cancri e,NaN,False
4275,Europa,False,D/297/P,NaN,NaN,False


In [ ]:
test_dataset.CryoSleep.replace(True, 1, inplace = True)
test_dataset.CryoSleep.replace(False, 0, inplace = True)

test_dataset.VIP.replace(False, 0, inplace = True)
test_dataset.VIP.replace(True, 1, inplace = True)

In [ ]:
test_dataset.Age = test_dataset.Age.fillna(test_dataset.Age.mean())

In [ ]:
test_dataset

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP
0,Earth,1.0,G/3/S,TRAPPIST-1e,27.000000,0.0
1,Earth,0.0,F/4/S,TRAPPIST-1e,19.000000,0.0
2,Europa,1.0,C/0/S,55 Cancri e,31.000000,0.0
3,Europa,0.0,C/1/S,TRAPPIST-1e,38.000000,0.0
4,Earth,0.0,F/5/S,TRAPPIST-1e,20.000000,0.0
...,...,...,...,...,...,...
4272,Earth,1.0,G/1496/S,TRAPPIST-1e,34.000000,0.0
4273,Earth,0.0,NaN,TRAPPIST-1e,42.000000,0.0
4274,Mars,1.0,D/296/P,55 Cancri e,28.658146,0.0
4275,Europa,0.0,D/297/P,NaN,28.658146,0.0


In [ ]:
test_dataset["HomePlanet"].replace("Earth", 0, inplace = True)
test_dataset["HomePlanet"].replace("Europa", 1, inplace = True)
test_dataset["HomePlanet"].replace("Mars", 2, inplace = True)

test_dataset["HomePlanet"] = test_dataset["HomePlanet"].fillna(4)

In [ ]:
test_dataset

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP
0,0.0,1.0,G/3/S,TRAPPIST-1e,27.000000,0.0
1,0.0,0.0,F/4/S,TRAPPIST-1e,19.000000,0.0
2,1.0,1.0,C/0/S,55 Cancri e,31.000000,0.0
3,1.0,0.0,C/1/S,TRAPPIST-1e,38.000000,0.0
4,0.0,0.0,F/5/S,TRAPPIST-1e,20.000000,0.0
...,...,...,...,...,...,...
4272,0.0,1.0,G/1496/S,TRAPPIST-1e,34.000000,0.0
4273,0.0,0.0,NaN,TRAPPIST-1e,42.000000,0.0
4274,2.0,1.0,D/296/P,55 Cancri e,28.658146,0.0
4275,1.0,0.0,D/297/P,NaN,28.658146,0.0


In [ ]:
test_dataset["Destination"] = test_dataset["Destination"].fillna("[0]")

destination_values = test_dataset["Destination"].unique()
print(destination_values)

for destination_value in destination_values:
  test_dataset.Destination.replace(destination_value, np.where(destination_values == destination_value)[0][0], inplace = True)

test_dataset

['TRAPPIST-1e' '55 Cancri e' 'PSO J318.5-22' '[0]']


,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP
0,0.0,1.0,G/3/S,0,27.000000,0.0
1,0.0,0.0,F/4/S,0,19.000000,0.0
2,1.0,1.0,C/0/S,1,31.000000,0.0
3,1.0,0.0,C/1/S,0,38.000000,0.0
4,0.0,0.0,F/5/S,0,20.000000,0.0
...,...,...,...,...,...,...
4272,0.0,1.0,G/1496/S,0,34.000000,0.0
4273,0.0,0.0,NaN,0,42.000000,0.0
4274,2.0,1.0,D/296/P,1,28.658146,0.0
4275,1.0,0.0,D/297/P,3,28.658146,0.0


In [ ]:
test_dataset["Cabin"] = test_dataset["Cabin"].fillna("[0]")
cabin_values = test_dataset.Cabin.unique()
cabin_values

for cabin_value in cabin_values:
  test_dataset.Cabin.replace(cabin_value, np.where(cabin_values == cabin_value)[0][0], inplace = True)


print(test_dataset)

      HomePlanet  CryoSleep  Cabin  Destination        Age  VIP
0            0.0        1.0      0            0  27.000000  0.0
1            0.0        0.0      1            0  19.000000  0.0
2            1.0        1.0      2            1  31.000000  0.0
3            1.0        0.0      3            0  38.000000  0.0
4            0.0        0.0      4            0  20.000000  0.0
...          ...        ...    ...          ...        ...  ...
4272         0.0        1.0   3262            0  34.000000  0.0
4273         0.0        0.0     15            0  42.000000  0.0
4274         2.0        1.0   3263            1  28.658146  0.0
4275         1.0        0.0   3264            3  28.658146  0.0
4276         0.0        1.0   3265            2  43.000000  0.0

[4277 rows x 6 columns]


In [ ]:
test_dataset[np.isnan(test_dataset) == True] = 0

In [ ]:
np.any(np.isnan(test_dataset["CryoSleep"]))

True

Predizione del dataset SENZA la ground_truth/variabile target, perchè ovviamente non la abbiamo.

In [ ]:
test_y_hat = model.predict(test_dataset)
test_y_hat_prob = model.predict_proba(test_dataset)

Dato che è una competizione Kaggle, salvataggio dei risulatati per l'invio

In [ ]:
submission_template = pd.read_csv("./drive/MyDrive/spaceshipTitanic/sample_submission.csv")
submission_template["Transported"] = test_y_hat_to_submission

submission_template.to_csv("./drive/MyDrive/spaceshipTitanic/submission.csv", index = False)